In [ ]:
#
#    Lambda Function code
#
#    Steps:
#    1.  In AWS create lambda function, provide name, Python 3, IAM execution role that has permissions to invoke 
#        Sagemaker endpoint
#    2.  Copy and paste below code into Lambda function.   In ENVIRONMENT VARIABLES place 'ENDPOINT_NAME' in "Key" with 
#        Sagemaker END_POINT want to invoke from Lambda in "Value"
#    3.  Setup Lambda test and run.  Data cannot have header and if more than one row being submitted then must 
#        have "/n" control character between rows of csv data
#    4.  Once complete Lambda setup and test proceed to creating an AWS API pointing to Lambda function with 
#        same name you used in #1 above
#    5.  Test the API with same data used in Lambda test
#    6.  If API test works, then deploy the AWS API.  Copy the url from clicking on method being used. 
#    7.  Either create a custom html web site that can call your deployed link (e.g. post, put, get, function) or 
#        you can use Postman.   Please note I found that you will get {"message":"Missing Authentication Token"} or {"message": "Forbidden"} message if you are not
#        using correct method (e.g. POST or GET) in html or Postman.  Also, the url may need a "/" at end of url after you 
#        copy the deployed AWS API url from AWS API method.
#   
#

import os
import io
import boto3
import json
import csv

# grab environment variables
ENDPOINT_NAME = os.environ['ENDPOINT_NAME']  ## args
runtime= boto3.client('runtime.sagemaker')

def lambda_handler(event, context):
    print("Received event: " + json.dumps(event, indent=2))
    
    data = json.loads(json.dumps(event))
    payload = data['data']
    print(payload)
    
    response = runtime.invoke_endpoint(EndpointName=ENDPOINT_NAME,
                                       ContentType='text/csv',
                                       Body=payload)
    print(response)
    result = json.loads(response['Body'].read().decode())
    print(result)
    
#   Custom formatting of data code can be placed below to make the return result more user friendly.

#   EXAMPLE:   pred = int(result['predictions'][0]['predicted_label'])
#              predicted_label = 'setosa iris' if pred == 1 else if 'not setosa iris'  
#              return predicted_label

#   Otherwise, simply return unformatted result.   Comment out below code if formatting and returning formatted result

    return {
        'statusCode': json.dumps('Success!'),
        'body': result
    }